# Process all file in folder





In [1]:
%load_ext autoreload
%autoreload 2

List the files to process in both conditions wt and sega

In [3]:
import pandas as pd

from pathlib import Path

# list files from the src folder
src = Path('/media/cephfs2/jparham/Joe for Jerome /')
dst = Path('./')

filelist = pd.concat((
    pd.DataFrame.from_records(
        [{'path':f, 'name':f.name, 'condition':'sega'}  for d in src.glob('*/*/Crop*') for f in d.glob('[!.]*.tif')]
    ),
    pd.DataFrame.from_records(
    [{'path':f, 'name':f.name, 'condition':'wt'}  for d in src.glob('Div*') for f in d.glob('[!.]*.tif')]
)),axis=0,ignore_index=True)

print(f"Number of files {len(filelist)}")
filelist.head()
filelist.to_csv('filelist.csv')

Number of files 62


## Process all file

In [ ]:
# from tqdm.notebook import tqdm
# tqdm.pandas()
# import motionquant as mq

# results_path = dst/Path('results.h5') # result
# if results_path.exists():
#     results_path.unlink()

# print(f"Saving results in {results_path}")

# df = []
# for filename in tqdm(filelist['path'].iloc):
#     try:
#         name = filename.stem        
#         img, mask, position, speed, diff, flow, rho, div, blob = mq.process(filename)
#         df.append(mq.record(name, img, mask, position, speed, diff, flow, rho, div, blob))
#         mq.save_result(results_path, name, img, mask, position, speed, diff, flow, rho, div, blob)
#     except Exception as e:
#         print(f"file '{filename}' could not be processed")
#         print(e)
#         pass
# df = pd.concat(df)


## Parallel processing

In [4]:
from dask.distributed import LocalCluster
cluster = LocalCluster()          # Fully-featured local Dask cluster
client = cluster.get_client()
cluster.scale(6)
cluster

LocalCluster(932d31c9, 'tcp://127.0.0.1:34851', workers=4, threads=12, memory=31.27 GiB)

In [10]:
import git
repo = git.Repo(search_parent_directories=True)
tag = repo.head.object.hexsha[-6:]
print(tag)

a58a45


In [5]:
import motionquant as mq
import dask
from dask.distributed import Lock
import torch
import traceback

def process(filename, results_path):
    name = filename.stem        
    img, mask, position, speed, diff, flow, rho, div, blob = mq.process(filename)
    lock = Lock('process-sufo')
    lock.acquire()
    mq.save_result(results_path, name, img, mask, position, speed, diff, flow, rho, div, blob)
    lock.release()
    df = mq.record(name, img, mask, position, speed, diff, flow, rho, div, blob)
    return df

def process_safe(filename, results_path):
    try:
        return process(filename, results_path)
    except Exception as e:
        print(f"file '{filename}' could not be processed")
        print(e)
        print(traceback.print_exc())
        pass


results_path = dst/Path(f'results-{tag}.h5') # result
if results_path.exists():
    results_path.unlink()

print(f"Saving results in file '{results_path}'")
tsk = [dask.delayed(process_safe)(filename, results_path) for filename in filelist['path'].iloc]
res = dask.compute(tsk)
df = pd.concat(res[0])

Saving results in file 'results.h5'


/home/jeromeb/miniconda3/envs/imaging/lib/python3.9/site-packages/skimage/measure/_regionprops.py:594: RuntimeWarning: invalid value encountered in double_scalars
  tuple(M[_get_element(axis)] / M0 for axis in range(self._ndim)))
/home/jeromeb/miniconda3/envs/imaging/lib/python3.9/site-packages/skimage/measure/_regionprops.py:594: RuntimeWarning: invalid value encountered in double_scalars
  tuple(M[_get_element(axis)] / M0 for axis in range(self._ndim)))
/home/jeromeb/miniconda3/envs/imaging/lib/python3.9/site-packages/skimage/measure/_regionprops.py:594: RuntimeWarning: invalid value encountered in double_scalars
  tuple(M[_get_element(axis)] / M0 for axis in range(self._ndim)))
/home/jeromeb/miniconda3/envs/imaging/lib/python3.9/site-packages/skimage/measure/_regionprops.py:594: RuntimeWarning: invalid value encountered in double_scalars
  tuple(M[_get_element(axis)] / M0 for axis in range(self._ndim)))
/home/jeromeb/miniconda3/envs/imaging/lib/python3.9/site-packages/skimage/measur

In [6]:
df.to_csv(dst/'results.csv')